<a href="https://colab.research.google.com/github/mucahitozkaya/app/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras.utils import to_categorical


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation,Dropout, BatchNormalization,MaxPool1D,LayerNormalization
from tensorflow.keras import Sequential
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC  # Import Support Vector Classifier (SVC)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Input
from logging import Logger
from tensorflow.keras.models import Model

#Read datas

In [50]:
train_final = pd.read_parquet("https://drive.google.com/uc?id=1ZeXb2EBMkP-nzULonq_hDnIxF5u9PwKp")
test_final = pd.read_parquet("https://drive.google.com/uc?id=1Blepx7rAAUXFLLNIL8gCI2g1IcNoMfTk")
submission_sample = pd.read_parquet("https://drive.google.com/uc?id=1KCcrx84tmG5biO0MjLmnFhvCMSC-U6ge")

In [51]:
df = train_final
df_test = test_final

In [ ]:
label_encoder = LabelEncoder()
df['target_encoded'] = label_encoder.fit_transform(df['target'])
df['target_encoded'].nunique()

112

In [ ]:
features_corr = df.loc[:, 'feature_0':'feature_24']
features_corr['target_encoded'] = df['target_encoded']
corr_matrix = features_corr.corr()

plt.figure(figsize=(24, 18))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Sütunlar Arasındaki Korelasyon")
plt.show()

In [ ]:
features_corr = df.loc[:, 'feature_25':'target_encoded']
features_corr = features_corr.drop(columns=['target'])
corr_matrix = features_corr.corr()

plt.figure(figsize=(24, 18))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Sütunlar Arasındaki Korelasyon")
plt.show()

In [52]:
df['target_cleaned'] = df['target'].apply(lambda x: ','.join([value.strip() for value in x.split(',')]))

df[['target1', 'target2','target3']] = df['target_cleaned'].str.split(',', expand=True)

df.drop(columns=['target','target_cleaned'], inplace=True)
df

,id,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,...,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target1,target2,target3
0,5beefd4d2bf4a4767e0df8108,10,5245.571,981.182,205.948,VODAFONE TR,Apple,-1.197737,1.113360,-1.123334,...,2.645719,-1.023478,1.658986,-1.559406,-2.161336,30,58,menu2,menu4,menu5
1,867285b116c063d5a8482f5be,10,5184.876,557.650,487.587,TURKCELL,samsung,-2.336352,2.567766,-0.494908,...,-0.021547,-0.195770,2.775513,-0.318980,-4.291473,21,45,menu7,menu8,menu4
2,c82a7cbd2e00d9b66c06bcadc,10,3835.618,3275.128,43.806,TURK TELEKOM,Redmi,-2.561455,2.061736,-0.184511,...,1.673868,0.631790,1.293131,-2.230909,-2.383524,19,61,menu2,menu8,menu4
3,f2d2b25073ccc298eced86897,10,3532.544,154.509,64.724,TURKCELL,samsung,-2.529918,3.358050,-0.851366,...,1.863495,0.213170,1.029710,-1.142185,-4.466191,2,41,menu6,menu2,menu1
4,7818c92a58af0f2cb7c361738,10,3344.192,787.896,715.115,VODAFONE TR,samsung,-2.922361,2.096124,0.060796,...,1.024499,-0.186423,-0.061626,-1.462175,-2.371206,23,85,menu6,menu2,menu8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,2e54f32ced9fae6ef802ceaa2,12,44.397,43.425,41.678,O2 - DE,HUAWEI,-1.531534,2.596604,0.340233,...,0.044385,0.863442,0.761354,1.073711,-3.560019,29,68,menu2,menu8,menu4
94045,6aa4ff0f6cc5ef4c2980b2862,12,44.331,43.977,40.620,TURKCELL,samsung,-1.268987,2.300487,0.231711,...,2.406462,-0.340132,-0.893553,-2.061401,-1.908158,25,43,menu9,menu2,menu5
94046,fa842185a0edd210845b78308,12,44.142,43.591,41.736,VODAFONE TR,Apple,-1.950039,2.805681,0.438200,...,0.449427,0.445361,-0.495204,-0.955097,-3.843092,4,25,menu6,menu2,menu4
94047,db8f55b8499f8d8c05148240e,12,43.963,43.350,40.862,TURK TELEKOM,Redmi,-2.389140,2.358281,0.683524,...,0.864857,1.284815,-0.100755,-1.056479,-1.452037,25,60,menu6,menu8,menu4


In [43]:
df['target1'].value_counts()


menu6    59439
menu9    15906
menu2    13943
menu3     2996
menu7     1146
menu8      367
menu1      111
menu4      105
menu5       36
Name: target1, dtype: int64

In [10]:
df['target1'].unique()

array(['menu2', 'menu7', 'menu6', 'menu9', 'menu3', 'menu8', 'menu1',
       'menu4', 'menu5'], dtype=object)

In [53]:
df['id'].nunique()
df.drop(columns=['carrier','devicebrand'], inplace=True)

In [39]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94049 entries, 0 to 94048
Data columns (total 58 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           94049 non-null  object 
 1   month        94049 non-null  int64  
 2   n_seconds_1  94049 non-null  float64
 3   n_seconds_2  94049 non-null  float64
 4   n_seconds_3  94049 non-null  float64
 5   feature_0    94049 non-null  float64
 6   feature_1    94049 non-null  float64
 7   feature_2    94049 non-null  float64
 8   feature_3    94049 non-null  float64
 9   feature_4    94049 non-null  float64
 10  feature_5    94049 non-null  float64
 11  feature_6    94049 non-null  float64
 12  feature_7    94049 non-null  float64
 13  feature_8    94049 non-null  float64
 14  feature_9    94049 non-null  float64
 15  feature_10   94049 non-null  float64
 16  feature_11   94049 non-null  float64
 17  feature_12   94049 non-null  float64
 18  feature_13   94049 non-null  float64
 19  feat

#Call Back

In [54]:
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime

# eğitim sırasında en iyi model ağırlıklarını kayıt edelim ve
# eğitim performansını TensorBoard ile izleyelim

filepath="weights-improvement-{epoch:02d}-{val_accuracy:04f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
logdir="/content/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
callbacks_list = [checkpoint,tensorboard_callback]


#Preprocess

In [55]:
label_encoder = LabelEncoder()
df['id'] = label_encoder.fit_transform(df['id'])
df['target1'] = label_encoder.fit_transform(df['target1'])
df['target2'] = label_encoder.fit_transform(df['target2'])
df['target3'] = label_encoder.fit_transform(df['target3'])



In [49]:
df['target3'].value_counts()


2    26029
6    19460
3    18868
0    16538
7     5905
4     4236
1     2836
5      177
Name: target3, dtype: int64

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94049 entries, 0 to 94048
Data columns (total 58 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           94049 non-null  int64  
 1   month        94049 non-null  int64  
 2   n_seconds_1  94049 non-null  float64
 3   n_seconds_2  94049 non-null  float64
 4   n_seconds_3  94049 non-null  float64
 5   feature_0    94049 non-null  float64
 6   feature_1    94049 non-null  float64
 7   feature_2    94049 non-null  float64
 8   feature_3    94049 non-null  float64
 9   feature_4    94049 non-null  float64
 10  feature_5    94049 non-null  float64
 11  feature_6    94049 non-null  float64
 12  feature_7    94049 non-null  float64
 13  feature_8    94049 non-null  float64
 14  feature_9    94049 non-null  float64
 15  feature_10   94049 non-null  float64
 16  feature_11   94049 non-null  float64
 17  feature_12   94049 non-null  float64
 18  feature_13   94049 non-null  float64
 19  feat

In [ ]:

# Katogorik verilerin one hot encoding yapılarak numerik hale getirilmesi
object_list = []
for column in df.columns:
     if df[column].dtype == 'object':
        object_list.append(column)
try:
  object_list.remove('id')
  df = df.drop(columns=['id'])
except:
  pass

df1 = pd.get_dummies(df, columns=object_list)

df1

,month,n_seconds_1,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,devicebrand_iBRIT,devicebrand_lge,devicebrand_meizu,devicebrand_motorola,devicebrand_nubia,devicebrand_realme,devicebrand_reeder,devicebrand_samsung,devicebrand_vivo,devicebrand_xiaomi
0,10,5245.571,-1.197737,1.113360,-1.123334,-0.263580,2.161242,2.651375,0.810021,1.516175,...,0,0,0,0,0,0,0,0,0,0
1,10,5184.876,-2.336352,2.567766,-0.494908,0.949101,3.567557,3.357848,0.434091,0.885814,...,0,0,0,0,0,0,0,1,0,0
2,10,3835.618,-2.561455,2.061736,-0.184511,1.062306,4.197788,1.551181,-0.596218,-0.618501,...,0,0,0,0,0,0,0,0,0,0
3,10,3532.544,-2.529918,3.358050,-0.851366,1.643876,2.849205,3.887427,1.854521,0.988186,...,0,0,0,0,0,0,0,1,0,0
4,10,3344.192,-2.922361,2.096124,0.060796,-1.487557,3.224788,2.091947,-0.992961,0.686043,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,12,44.397,-1.531534,2.596604,0.340233,-1.149720,3.308406,2.995904,1.117685,0.971643,...,0,0,0,0,0,0,0,0,0,0
94045,12,44.331,-1.268987,2.300487,0.231711,0.741582,2.365813,2.031927,1.195348,0.209732,...,0,0,0,0,0,0,0,1,0,0
94046,12,44.142,-1.950039,2.805681,0.438200,2.976427,4.001829,3.051181,0.431400,1.755406,...,0,0,0,0,0,0,0,0,0,0
94047,12,43.963,-2.389140,2.358281,0.683524,0.234449,4.070453,1.709853,1.140759,0.187314,...,0,0,0,0,0,0,0,0,0,0


In [56]:

df_test['id'] = label_encoder.fit_transform(df_test['id'])
df_test.drop(columns=['carrier','devicebrand'], inplace=True)

In [73]:
df_test

,id,month,n_seconds_1,n_seconds_2,n_seconds_3,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,2155,1,6893.544,246.854,242.636,-1.723524,3.216489,-1.138474,2.026997,2.241670,...,-1.094519,-1.217407,-4.280456,1.512240,-2.306445,2.066388,0.844927,-1.026193,18,58
1,9211,1,4481.065,740.209,263.860,-0.417275,2.024433,0.102952,-1.634336,3.621519,...,1.806486,-3.477517,-2.064966,1.499805,1.284697,0.189269,-1.563224,-1.901654,3,35
2,3609,1,4340.702,2742.163,318.700,-2.943294,2.769536,0.734942,1.681471,3.229447,...,1.759080,-2.038839,-2.067219,2.141083,0.055355,0.084739,-1.009925,-2.058473,7,50
3,2440,1,4129.666,181.397,155.423,-2.346902,2.684752,0.168206,-1.072321,4.971480,...,2.171847,-0.925040,-1.484278,0.666036,0.911519,0.616167,0.092304,-1.874706,22,47
4,91,1,3903.944,126.133,100.060,-1.745354,2.355863,0.318961,-0.570734,4.056542,...,-0.373413,-0.015773,-2.961445,1.301413,1.375090,-0.107355,0.924390,-1.606419,29,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11950,5559,1,47.726,40.879,40.647,-2.952961,1.352361,-0.396846,-0.353791,3.740440,...,0.333993,-0.533786,-1.779781,0.228962,0.403641,-0.906588,-0.788647,-0.627803,18,45
11951,4508,1,47.471,44.745,41.040,-1.945921,2.003634,-0.300293,-0.423918,3.141109,...,1.468012,-1.345186,-1.067598,1.685431,0.879146,0.206344,-1.623952,-2.496700,20,44
11952,8094,1,47.403,45.832,43.932,-1.596234,3.165674,-1.520392,-0.195639,3.962691,...,1.096878,-0.558995,-3.029989,1.352324,0.351689,-0.326579,-0.382071,-2.995726,17,34
11953,5891,1,46.044,43.592,40.735,-1.984582,2.382992,-1.060896,0.930493,3.623143,...,1.006593,-2.548628,-2.803372,1.091970,0.436466,0.235511,-1.625251,-2.354214,26,58


In [57]:
X = df.drop(["target1", "target2", "target3"], axis=1)
y1 = df[["target1"]]
y2 = df[["target2"]]
y3 = df[["target3"]]


# Veriyi eğitim ve test setlerine bölelim
X_train, X_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(
    X, y1, y2, y3, test_size=0.2, random_state=42
)

# Modeli oluşturma
input_layer = Input(shape=(55,))
embedding_layer = Embedding(input_dim=5000, output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding_layer)

# İlk çıkış
output1 = Dense(9, activation='softmax', name='output1')(lstm_layer)

# İkinci çıkış
output2 = Dense(9, activation='softmax', name='output2')(lstm_layer)

# Üçüncü çıkış
output3 = Dense(9, activation='softmax', name='output3')(lstm_layer)

# Modeli oluştur
model = Model(inputs=input_layer, outputs=[output1, output2, output3])

# Modeli derle
model.compile(optimizer='adam',
              loss={'output1': 'sparse_categorical_crossentropy',
                    'output2': 'sparse_categorical_crossentropy',
                    'output3': 'sparse_categorical_crossentropy'},
              metrics=['accuracy'])

# Modeli eğit
model.fit(X_train, {'output1': y1_train, 'output2': y2_train, 'output3': y3_train},
          validation_data=(X_test, {'output1': y1_test, 'output2': y2_test, 'output3': y3_test}),
          epochs=10, batch_size=32)

# Modeli değerlendir
loss_and_metrics = model.evaluate(X_test, [y1_test, y2_test, y3_test])
loss1 = loss_and_metrics[0]
accuracy1 = loss_and_metrics[1]
loss2 = loss_and_metrics[2]
accuracy2 = loss_and_metrics[3]
loss3 = loss_and_metrics[4]
accuracy3 = loss_and_metrics[5]

Epoch 1/100
2352/2352 [==============================] - 29s 11ms/step - loss: 4.1429 - output1_loss: 1.0879 - output2_loss: 1.3139 - output3_loss: 1.7411 - output1_accuracy: 0.6326 - output2_accuracy: 0.6135 - output3_accuracy: 0.2754 - val_loss: 4.1064 - val_output1_loss: 1.0813 - val_output2_loss: 1.2960 - val_output3_loss: 1.7292 - val_output1_accuracy: 0.6279 - val_output2_accuracy: 0.6194 - val_output3_accuracy: 0.2768
Epoch 2/100
2352/2352 [==============================] - 22s 9ms/step - loss: 4.1101 - output1_loss: 1.0765 - output2_loss: 1.3047 - output3_loss: 1.7290 - output1_accuracy: 0.6330 - output2_accuracy: 0.6138 - output3_accuracy: 0.2784 - val_loss: 4.1092 - val_output1_loss: 1.0832 - val_output2_loss: 1.2974 - val_output3_loss: 1.7286 - val_output1_accuracy: 0.6279 - val_output2_accuracy: 0.6194 - val_output3_accuracy: 0.2806
Epoch 3/100
2352/2352 [==============================] - 20s 9ms/step - loss: 4.1047 - output1_loss: 1.0757 - output2_loss: 1.3029 - output3_lo

In [58]:



# Tahminleri alın
predictions = model.predict(df_test)

# Her bir çıkış için tahminleri ayırın
predicted_output1 = predictions[0]
predicted_output2 = predictions[1]
predicted_output3 = predictions[2]


# Tahmin sonuçlarını tam sayılara dönüştürme
predicted_output1 = np.argmax(predicted_output1, axis=1)
predicted_output2 = np.argmax(predicted_output2, axis=1)
predicted_output3 = np.argmax(predicted_output3, axis=1)


374/374 [==============================] - 1s 3ms/step


In [59]:
for i in range(len(predicted_output1)):
  binary_array = [0] * 9

  binary_array[predicted_output1[i]] = 1
  binary_array[predicted_output2[i]] = 1
  binary_array[predicted_output3[i]] = 1

  binary_string = ''.join(map(str, binary_array))
  submission_sample.at[i, 'target'] = binary_string


In [60]:
submission_sample

,id,target
0,2e6105f5911256f4f6c4813ed,110001000
1,c56ad71dae0a5dbd3e7d36adc,010001010
2,4d02ea175f6581f0c6385311f,011001000
3,3412d27a86c286ba078fa935c,010000101
4,0203b561f6f7e10eafa46eefa,011001000
...,...,...
11950,7687113f46112edf4f56666ee,011001000
11951,5ff8eb7a06fd48b60dbc04f34,011001000
11952,ac23a7b9ad3e5d61e738c854b,011001000
11953,7da05018634ea2eee4b122756,011001000


In [61]:
# Define the file path for the CSV file
csv_file_path = 'submission_sample.csv'

# Export the DataFrame to CSV
submission_sample.to_csv(csv_file_path, index=False)


In [6]:
type(dff['target'])

pandas.core.series.Series

In [9]:
dff = pd.read_csv("/content/submission_sample2.csv")
dff = dff.astype(str)


def count_non_three_ones(df, column_name='target'):
    count = 0
    for target in df[column_name]:
        if target.count('1') != 3:
            count += 1
            print(target)


count_non_three_ones(dff)

10100000
10001000
10001000
1100
10100000
10100000
10100000
101000
10100000
10100000
10100000
10001000
10100000
10100000
1100
101000
10100000
11000000
10000100
10000100
10100000
101000
10100000
10100000
10001000
10100000
10100000
100001000
10100000
10100000
101000
10100000
10100000
10100000
10001000
10001000
100001
10100000
10100000
10000100
10100000
1100
101000
10001000
10100000
101000
10100000
10100000
10100000
10100000
10100000
10100000
100001000
10100000
101000
10100000
10000100
10100000
10100000
10001000
10100000
10100000
1010
10001000
10001000
101000
10100000
101000
10100000
10100000
10100000
101000
10000001
10100000
1001000
10100000
10100000
10100000
10001000
101000
10001000
1000001
101000
10000100
10000100
10100000
100001000
10001000
100001
10100000
10001000
101000
10001000
10100000
11000000
1001000
101000
10100000
10000100
101000
101000
10100000
10100000
10100000
10001000
10001000
10100000
10100000
10100000
10001000
101000
10001000
10000100
101000
10100000
10100000
10100000
101

In [53]:
def build_lstm_model(neurons, input_shape, lr, num_classes, activation, loss):
    try:
        #Logger.info("Building LSTM Model...")
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
        inputs = tf.keras.layers.Input(shape=input_shape)
        x = tf.keras.layers.Embedding(input_dim=5000, output_dim=10)(inputs)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(neurons))(x)
        # Define separate output layers for each target

        output_1 = tf.keras.layers.Dense(num_classes, activation=activation, name="output_1")(x)
        output_2 = tf.keras.layers.Dense(num_classes, activation=activation, name="output_2")(x)
        output_3 = tf.keras.layers.Dense(num_classes, activation=activation, name="output_3")(x)
        # Define the model with multiple outputs
        model = tf.keras.Model(inputs=inputs, outputs=[ output_1, output_2, output_3])
        model.compile(loss=loss, optimizer=opt, metrics=["sparse_categorical_accuracy"])
        #Logger.debug("LSTM Model Built...")
        return model
    except Exception as e:
        print("Exception while building model:", e)
        raise


X = df.drop(["target1", "target2", "target3"], axis=1)
y = df[["target1", "target2", "target3"]]

y = np.array(y)

(X, x_test, y, y_test) = train_test_split(X, y, random_state=100, test_size=0.1, shuffle=True)


(x_train, x_val, y_train, y_val) = train_test_split(X, y, random_state=100, test_size=0.1, shuffle=True)

#Variables
neurons = 100
input_shape = 55
lr = 0.001
activation = 'softmax'
loss = 'sparse_categorical_crossentropy'
num_classes = 9 #number of unique classes per label

model = build_lstm_model(neurons, input_shape, lr, num_classes, activation, loss)

early_stopings = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=2, verbose=1, mode="auto")
callbacks = [early_stopings]

trainHistrory = model.fit(
    x=x_train,
    y=[y_train[:, 0], y_train[:, 1], y_train[:, 2]],
    validation_data=(x_val, [y_val[:, 0], y_val[:, 1], y_val[:, 2]]),
    epochs= 10,
    batch_size=32,
    verbose=1,
    callbacks=callbacks,
)

#Evaluate on test data
# x_test = test_df.drop(["TARGET_0", "TARGET_1", "TARGET_2", "TARGET_3"], axis=1)
# y_test = test_df[["TARGET_0", "TARGET_1", "TARGET_2", "TARGET_3"]]

# y_test = np.array(y_test)

test_accuracy = model.evaluate([x_test], [y_test[:, 0], y_test[:, 1], y_test[:, 2]])

Epoch 1/10
2381/2381 [==============================] - 37s 14ms/step - loss: 4.1205 - output_1_loss: 1.0842 - output_2_loss: 1.3065 - output_3_loss: 1.7299 - output_1_sparse_categorical_accuracy: 0.6323 - output_2_sparse_categorical_accuracy: 0.6151 - output_3_sparse_categorical_accuracy: 0.2797 - val_loss: 4.0516 - val_output_1_loss: 1.0821 - val_output_2_loss: 1.2863 - val_output_3_loss: 1.6833 - val_output_1_sparse_categorical_accuracy: 0.6305 - val_output_2_sparse_categorical_accuracy: 0.6156 - val_output_3_sparse_categorical_accuracy: 0.3259
Epoch 2/10
2381/2381 [==============================] - 29s 12ms/step - loss: 4.0265 - output_1_loss: 1.0703 - output_2_loss: 1.2819 - output_3_loss: 1.6743 - output_1_sparse_categorical_accuracy: 0.6326 - output_2_sparse_categorical_accuracy: 0.6154 - output_3_sparse_categorical_accuracy: 0.3295 - val_loss: 4.0047 - val_output_1_loss: 1.0756 - val_output_2_loss: 1.2753 - val_output_3_loss: 1.6538 - val_output_1_sparse_categorical_accuracy: 0

In [40]:
predicted_labels = [np.argmax(pred, axis=-1) for pred in predictions]

In [43]:
predicted_labels[0]

array([5, 5, 1, ..., 5, 5, 5])

In [31]:

X, y = df.drop(columns=['target1', 'target2', 'target3']), df[['target1', 'target2', 'target3']]
# Veri standartlaştırma
scaler = StandardScaler()
data_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(data_scaled,y, test_size=0.1, random_state=42)




In [32]:
y_train

,target1,target2,target3
65710,5,1,6
82341,8,4,0
33587,8,1,4
323,5,1,2
3840,5,6,2
...,...,...,...
6265,5,1,0
54886,5,1,2
76820,5,1,6
860,8,6,2


In [ ]:
X_train

array([[ 0.94010876,  0.02149488,  0.28148241, ..., -0.11106506,
        -0.06259   , -0.45116505],
       [ 0.94010876, -0.55099891,  1.49522509, ..., -0.11106506,
        -0.06259   ,  2.21648375],
       [-0.42545704, -0.33933236,  1.58461666, ..., -0.11106506,
        -0.06259   ,  2.21648375],
       ...,
       [ 0.94010876, -0.40618411, -0.05337361, ..., -0.11106506,
        -0.06259   , -0.45116505],
       [-1.79102283,  0.98709599, -0.85689147, ..., -0.11106506,
        -0.06259   ,  2.21648375],
       [-0.42545704,  2.64279972, -0.88064255, ..., -0.11106506,
        -0.06259   , -0.45116505]])

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense

# Input layer'ları oluşturun
input1 = Input(shape=(max_sequence_length,))
input2 = Input(shape=(max_sequence_length,))
input3 = Input(shape=(max_sequence_length,))

# Embedding katmanlarını oluşturun
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length)

# LSTM katmanlarını oluşturun
lstm_layer = LSTM(64, return_sequences=True)

# Her sütun için farklı çıkış katmanlarını oluşturun
output1 = Dense(3, activation='softmax', name='output1')(lstm_layer(embedding_layer(input1)))
output2 = Dense(3, activation='softmax', name='output2')(lstm_layer(embedding_layer(input2)))
output3 = Dense(3, activation='softmax', name='output3')(lstm_layer(embedding_layer(input3)))

# Modeli oluşturun
model = Model(inputs=[input1, input2, input3], outputs=[output1, output2, output3])

# Modeli derleyin
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitin
model.fit([X_train_padded, X_train_padded, X_train_padded], [y_train1, y_train2, y_train3], validation_data=([X_test_padded, X_test_padded, X_test_padded], [y_test1, y_test2, y_test3]), epochs=10, batch_size=64)


In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(8, activation="relu", name="layer1"),
        layers.Dense(6, activation="relu", name="layer2"),
         LayerNormalization(axis=1),
        layers.Dense(3, activation="relu", name="layer3"),
        layers.Dense(3, activation="relu", name="layer4"),
        Dropout(0.4),
        layers.Dense(6, activation="relu", name="layer5"),
        Dropout(0.4),
         LayerNormalization(axis=1),
        layers.Dense(8, activation="relu", name="layer6"),
        layers.Dense(9, name="layer7"),
    ]
)

# Modeli Derleme
model.compile(loss='categorical_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy'])

history = model.fit(x=X_train,y=y_train,
                        epochs=150,
                        validation_data = (X_test,y_test),
                       callbacks=callbacks_list
                        )

Epoch 1/150
2641/2646 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 1: val_accuracy improved from -inf to 0.00096, saving model to weights-improvement-01-0.000957.hdf5
2646/2646 [==============================] - 24s 6ms/step - loss: nan - accuracy: 0.0012 - val_loss: nan - val_accuracy: 9.5694e-04
Epoch 2/150
  21/2646 [..............................] - ETA: 13s - loss: nan - accuracy: 0.0030    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2644/2646 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 2: val_accuracy did not improve from 0.00096
2646/2646 [==============================] - 15s 6ms/step - loss: nan - accuracy: 0.0012 - val_loss: nan - val_accuracy: 9.5694e-04
Epoch 3/150
2638/2646 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 3: val_accuracy did not improve from 0.00096
2646/2646 [==============================] - 15s 6ms/step - loss: nan - accuracy: 0.0012 - val_loss: nan - val_accuracy: 9.5694e-04
Epoch 4/150
2642/2646 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 4: val_accuracy did not improve from 0.00096
2646/2646 [==============================] - 15s 6ms/step - loss: nan - accuracy: 0.0012 - val_loss: nan - val_accuracy: 9.5694e-04
Epoch 5/150
2646/2646 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0012
Epoch 5: val_accuracy did not improve from 0.00096
2646/2646 [=================